In [ ]:
from XAJ import XinAnJiang
# the XAJ model can refer to 
from sko.DE import DE
from sko.PSO import PSO

In [ ]:
"""
core of XinAnJiang
ini_soil_moisture: Initial soil moisture content
evapor_single_period: Evaporation per time period
runoff_generation_single_period: Single-session abortion
different_sources: sub-source calculation
uh_forecast: Sink calculations using time period unit lines
route_linear_resourvior: Linear reservoir slope confluence
"""

import numpy as np
import pandas as pd

def ini_soil_moisture(xaj_params,w0_initial,day_precip,day_evapor):
    
    """
    Objective
    ---------
    Based on the initial soil water content, rainfall, and evaporation,
    derive the soil water content, evaporation, and flow rate for each time period.

    Parameters
    ----------
    xaj_params：model parameters
    w0_initial：Initial soil moisture content
    day_precip：Daily precipitation in the basin
    day_evapor：Daily evaporation from the watershed

    Returns
    -------
    w0：Pre-basin soil moisture content
    """
    
    w0 = w0_initial
    eu_s, el_s, ed_s, wu_s, wl_s, wd_s, r_s =[], [], [], [], [], [], []
    for i in range(day_evapor.size):
        eu, el, ed, wu, wl, wd, r = evapor_single_period(xaj_params, w0, day_precip[i], day_evapor[i])
        w0 = pd.DataFrame({'Value':[wu,wl,wd]},index=['wu','wl','wd'])
        eu_s.append(eu)
        el_s.append(el)
        ed_s.append(ed)
        wu_s.append(wu)
        wl_s.append(wl)
        wd_s.append(wd)
        r_s.append(r)
    result=pd.DataFrame([eu_s, el_s, ed_s, wu_s, wl_s, wd_s, r_s],index=['eu', 'el', 'ed', 'wu', 'wl', 'wd', 'r'])
    return result.T


def evapor_single_period(evapor_params, initial_conditions, precip, evapor):
    
    """
    Objective
    ---------
    Calculation of evaporation per time period

    Method
    ------
    Three-layer evaporation pattern, not considering impervious area

    Parameters
    ----------
    evapor_params: Evaporation model parameters
    initial_conditions: Initial soil moisture content conditions
    precip: Average surface rainfall
    evapor: Evaporation

    Evapor-Parameters
    -----------------
    KC  : Evaporation capacity conversion factor
    WUM : Upper Tension Water Storage Capacity
    WLM : Lower Tension Water Storage Capacity
    WM  : Average storage capacity of the basin
    C   : Deep evapotranspiration conversion factor

    Returns
    -------
    eu,el,ed: Triple vaporization
    wu,wl,wd: Triple moisture content
    """

    #读取计算参数
    k = evapor_params.loc['KC','Value']
    wum = evapor_params.loc['WUM','Value']
    wlm = evapor_params.loc['WLM','Value']
    c = evapor_params.loc['C','Value']
    wm = evapor_params.loc['WM','Value']

    #evapor为观测蒸发，e计算为实际蒸发
    ep = k * evapor
    p = precip

    #读取初始土壤含水量参数
    wu0 = initial_conditions.loc['wu','Value']
    wl0 = initial_conditions.loc['wl','Value']
    wd0 = initial_conditions.loc['wd','Value']

    #计算深层张力水蓄水容量
    wdm = wm - wum - wlm

    #三层蒸发模式计算
    if wu0 + p >= ep:
        eu = ep
        el = 0
        ed = 0
    else:
        eu=round(wu0+p,1)
        if wl0 >= c * wlm:
            el=round((ep - eu) * wl0 / wlm, 1)
            ed = 0
        if wl0 >= c * (ep - eu) and wl0 < c * wlm:
            eu = round(wu0 + p, 1)
            el = round(c * (ep - eu), 1)
            ed = 0
        if wl0 < c * (ep - eu):
            eu = round(wu0 + p, 1)
            el = wl0
            ed = round(c * (ep - eu) - el, 1)
    if p - ep > 0:
        r = runoff_generation_single_period(evapor_params, initial_conditions, precip, evapor)
        wu = wu0 + p - eu - r
        wl = wl0 - el
        wd = wd0 - ed
        if wu > wum:
            wl = wl + wu - wum
            wu = wum
        if wl > wlm:
            wd = wd + wl - wlm
            wl = wlm
        if wu < 0.001:
            wu = 0
        if wl < 0.001:
            wl = 0 
        if wd < 0.001:
            wd = 0
    else:
        r = 0
        wu = wu0 + p - eu
        wl = wl0 - el
        wd = wd0 - ed
    return eu, el, ed, wu, wl, wd, r


def runoff_generation_single_period(gene_params, initial_conditions, precip, evapor):
    """
    Objective
    ---------
    Calculation of streamflow production in the time zone watershed

    Method
    ------
    model of saturation-exceed runoff generation

    Parameters
    ----------
    gene_params: runoff generation model parameter
    initial_conditions: Conditions for the initial calculation of the time period
    precip: Average surface rainfall for the period
    evapor: Evapotranspiration from the watershed for the time period

    Gene-Parameters
    ---------------
    wm: Basin average tensional water storage capacity
    B : Tension water storage capacity area distribution curve square times 

    Returns
    -------
    runoff: Basin hourly flow production
    """
    b = gene_params.loc['B','Value']
    wm = gene_params.loc['WM','Value']

    wu0 = initial_conditions.loc['wu','Value']
    wl0 = initial_conditions.loc['wl','Value']
    wd0 = initial_conditions.loc['wd','Value']
    w0 = wu0 + wl0 + wd0

    pe = precip - evapor
    wmm = wm * (1 + b)
    a = wmm * (1 - (1 - w0 / wm) ** (1 / (1 + b)))

    if a + pe < wmm:
        r = round(pe + w0 - wm + wm * (1 - (a + pe) / wmm) ** (b + 1), 1)
    else:
        r = round(pe + w0 - wm, 1)

    return r


def different_sources(diff_source_params, initial_conditions, precip, evapor, runoff):
    """
    Objective
    ---------
    sub-source calculation

    Method
    ------
    Water Tank Model Three Water Source Division

    Parameters
    ----------
    diff_source_params: Parameters required for sub-source calculations
    initial_conditions: Initial conditions
    precips: precipitation
    evapors: evapotranspiration
    runoffs: runoff

    Diff-Source-Params
    ------------------
    k : Basin evapotranspiration capacity conversion factor
    sm: Topsoil free water storage
    ex: Free Water Storage Capacity Area Allowance Curve Index
    ki: Sunrise coefficient for free-water impoundment reservoir loam mid-stream
    kg: Groundwater sunrise coefficient for free-water impoundments

    Returns
    -------
    Diff_source: Sub-source calculations
    """

    # 取参数值
    k = diff_source_params.loc['KC','Value']
    sm = diff_source_params.loc['SM','Value']
    ex = diff_source_params.loc['EX','Value']
    ki = diff_source_params.loc['KI','Value']
    #KI+KG=0.7，即雨止到壤中流止的时间为3天
    kg = 0.7 - ki

    # 为便于后面计算，这里以数组形式给出s和fr
    s_s = []#自由水蓄水库自由水蓄量
    fr_s = []#产流面积

    # 取初始值
    s0 = initial_conditions.loc['S0','Value']
    fr0 = initial_conditions.loc['FR0','Value']

    # 流域最大点自由水蓄水容量深
    ms = sm * (1 + ex)

    rs_s, ri_s, rg_s = [], [], []
    for i in range(precip.size):
        #上一时段的产流面积FR0，自由水蓄水量S0
        if i == 0:
            fr0 = fr0
            s0 = s0
        else:
            fr0 = fr_s[i-1]
            s0 = s_s[i-1]
        #本时段计算
        p = precip[i]
        e = k * evapor[i]
        if runoff[i] < 0:
            rs = 0
            ri = s0 * fr0 * ki
            ri = round(ri , 3)
            rg = s0 * fr0 * kg
            rg = round(rg , 3)
            fr=fr0
            if fr <= 0:
                fr = 0.001
            if fr > 0:
                fr = 1
            s = s0 * (1 - ki - kg)
            if s < 0:
                s = 0
            if s > sm:
                s = sm
            if ri < 0.01:
                ri = 0
            if rg < 0.01:
                rg = 0
        else:
            pe = p - e
            fr = runoff[i] / pe
            if fr <= 0:
                fr = 0.01
            if fr > 1:
                fr = 1
            au = ms * (1 - ((1 - ((s0 * fr0) / (fr * sm))) ** (1 / (1 + ex))))
            if pe + au < ms:
                rs = fr * (pe + (s0 * fr0) / fr - sm + sm * ((1 - (pe + au) / ms) ** (1+ex)))
            else:
                rs = fr * (pe + s0 * fr0 / fr - sm)
            s = s0 * fr0 / fr + (runoff[i] - rs) / fr
            ri = ki * s * fr
            rg = kg * s * fr
            rs = round(rs , 3)
            ri = round(ri , 3)
            rg = round(rg , 3)
            s0 = s * (1 - ki - kg)
            if rs < 0.01:
                rs = 0
            if ri < 0.01:
                ri = 0
            if rg < 0.01:
                rg = 0
        fr_s.append(fr)
        s_s.append(s)
        rs_s.append(rs)
        ri_s.append(ri)
        rg_s.append(rg)
        diff_source=pd.DataFrame([rs_s, ri_s, rg_s], index=['rs', 'ri', 'rg'])
    return diff_source.T

def uh_forecast(runoffs, uh):
    
    """
    Sink calculations using time period unit lines
    
    Parameters
    ------------
    runoffs: water recharge from rainfall
    uh: Unit line values by time period

    Return
    ------------
    runoff hydrograph
    """
    
    q = np.convolve(runoffs, uh)
    return q

def route_linear_resourvior(route_params, basin_property, initial_conditions, rs, ri, rg):
    
    """
    Objective
    ---------
    Runoff concentration

    Method
    ------
    Linear reservoirs

    Parameters
    ----------
    route_params：Runoff concentration parameters
    basin_property：Basin attribute conditions
    initial_confitions：Initial conditions
    rs：Surface runoff
    ri：Soil runoff
    rg：Underground runoff 

    Route-Params
    ------------
    cs：Surface runoff recession factor
    ci：Soil runoff recession factor
    cg：Underground runoff recession factor

    Returns
    -------
    QT_Network Unit area total river network inflow
    """
    
    f = basin_property.loc['basin_area/km^2','Value']
    time_in = 24
    u = f / (3.6 * time_in)

    cs = route_params.loc['CS','Value']
    ci = route_params.loc['CI','Value']
    cg = route_params.loc['CG','Value']

    qs0=initial_conditions.loc['QS0','Value']
    qi0=initial_conditions.loc['QI0','Value']
    qg0=initial_conditions.loc['QG0','Value']

    qs_s, qi_s, qg_s = [], [], []
    
    qs_s.append(qs0)
    qi_s.append(qi0)
    qg_s.append(qg0)

    for i in range(len(rs)):
        if i == 0:
            qs0 = qs0
            qi0 = qi0
            qg0 = qg0
        else:
            qs0 = qs_s[i-1]
            qi0 = qi_s[i-1]
            qg0 = qg_s[i-1]

        qs = cs * qs0 + (1 - cs) * rs[i] * u
        qi = ci * qi0 + (1 - ci) * ri[i] * u
        qg = cg * qg0 + (1 - cg) * rg[i] * u

        if i == 0:
            qs_s[i] = qs
            qi_s[i] = qi
            qg_s[i] = qg
        else:
            qs_s.append(qs)
            qi_s.append(qi)
            qg_s.append(qg)

    qs_s=np.array(qs_s)
    qi_s=np.array(qi_s)
    qg_s=np.array(qg_s)
    qt_s = qs_s + qi_s + qg_s
    
    QT_Network=pd.DataFrame([qs_s,qi_s,qg_s,qt_s], index=['qs','qi','qg','qt'])
    return QT_Network.T

In [ ]:
def xaj_model(p):
    
    """
    Objective
    ---------
    Packaging the Xin'anjiang model with data readouts for optimized calculations    

    Parameters
    ----------
    K,B,C,WM,WUM,IM,SM,EX,KI,KSUM,CG,CI,CS,L : Parameters in XinAnJiang model
    P : Precipitation
    E : Evapotranspiration
    Q : Runoff
    time_index : time step index

    Route-Params
    ------------
    rp : simulated result from XAJ model
    y : metrics between observation and prediction
    y['确定性系数'] : Nash-Sutcliffe efficiency coefficient between observation and prediction

    Returns
    -------
    1 - y['确定性系数'][0] : Taking this value lower will achieving higher NSE
    """
    
    K,B,C,WM,WUM,IM,SM,EX,KI,KSUM,CG,CI,CS,L = p
    # p-parameters
    model = XinAnJiang(24,111054,K,B,C,WM,WUM,100-WUM,IM,SM,EX,KI,KSUM-KI,CG,CI,CS,L,base_flow = 0)
    
    data = read_csv('data/2001-2017Data(Mean).csv', header=0)
    
    data_t = data.iloc[:4968,:] # training data
    data_v = data.iloc[4969:,:] # validation data
    time_index = np.append(0, data_t['time'].values)
    
    P = data_t['Prec'].values
    E = data_t['Evp'].values
    Q = np.append(0, data_t['flow'].values)    
    time_index = np.append(0, data_v['time'].values)  
    
    model.simulate(P,E,time_index)
    rp = model.simulate_result
    y = model.error(Q)
    
    return 1 - y['确定性系数'][0] # taking this value lower will achieving higher NSE

In [ ]:
from sko.tools import set_run_mode
set_run_mode(xaj_model, 'multithreading')

pso = PSO(func=xaj_model, n_dim=14, pop=100, max_iter=100, w=0.8, c1=0.5, c2=0.5,
          lb=[1,   0.4,  0.08,  220,  10,  0.1,  10,  1.0,  0.2,   0.7,     0.9,   0.5,  0.3,  1],
          ub=[1.5, 0.6,  0.167, 300,  90,  0.7,  50,  1.5,  0.7,   0.8,    0.999,  0.8,  0.5, 15])
#              K,    B,    C,    WM,  WUM,  IM,   SM,  EX,   KI,   KSUM,    CG,    CI,   CS,   L

best_x, best_y = pso.run()
print('best_nse:', 1 - best_y)
tp = pd.DataFrame(data=None,columns=['K','B','C','WM','WUM','IM','SM','EX','KI','KI+KG','CG','CI','CS','L'])
tp.loc[1] = best_x
tp